# 딕셔너리와 집합

dict 형은 애플리케이션에서 널리 사용될 뿐만 아니라 파이썬 구현의 핵심 부분이기도 하다.
모듈 네임스페이스, 클래스 및 인스턴스 고성, 함수의 키워드 인수 등 핵심 부분에 딕셔너리가 사용되고 있다.
내장 함수들은 `__builtins__.__dict__`에 들어 있다.

중요한 역할을 맡고 있으므로 파이썬 dict 클래스는 상당히 최적화 되어있다. 파이썬의 고성능 딕셔너리 뒤에는 해시 테이블이라는 엔진이 있다.

집합도 해시 테이블을 이용해서 구현하므로, 이 장에서는 집합도 다룬다.

해시 테이블이 작동하는 방식을 알아야 딕셔너리와 집합을 최대로 활용할 수 있다.

In [3]:
my_dict = {}

import collections
isinstance(my_dict, collections.abc.Mapping)

# 함수 인수가 dict 형인지 검사하는 것보다 isinstance() 함수를 사용하는 것이 좋다. 다른 매핑형이 사용될 수도 있기 때문이다.

True

In [10]:
tt = (1, 2, (30, 40))
print(hash(tt))

tl = (1, 2, [30, 40])
# hash(tl)

tf = (1, 2, frozenset([30, 40]))
print(hash(tf))

-3907003130834322577
5149391500123939311


In [11]:
# dict을 구현하는 다양한 방법
a = dict(one=1, two=2, three=3)
print(a)
b = {'one': 1, 'two': 2, 'three': 3}
print(b)
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
print(c)
d = dict([('two', 2), ('one', 1), ('three', 3)])
print(d)
e = dict({'three': 3, 'one': 1, 'two': 2})
print(a == b == c == d == e)

{'one': 1, 'two': 2, 'three': 3}
{'one': 1, 'two': 2, 'three': 3}
{'one': 1, 'two': 2, 'three': 3}
{'two': 2, 'one': 1, 'three': 3}
True


### 지능형 딕셔너리
파이썬 2.7부터는 지넝형 리스트와 제너레이터 표현식 구문이 지능형 딕셔너리에도 적용된다. 지능형 딕셔너리는 모든 반복형 객체에서 키-값 쌍을 생성하믕로써 딕셔너리 객체를 만들 수 있다.

다음은 지능형 딕셔너리를 이용해서 동일한 튜플 리스트에서 딕셔너리 두 개를 생성하는 과정을 보여준다.

In [15]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States'),
    (62, 'Indonesia'),
    (55, 'Brazil'),
    (92, 'Pakistan'),
    (880, 'Bangladesh'),
    (234, 'Nigeria'),
    (7, 'Russia'),
    (81, 'Japan'),
]

country_code = {country: code for code , country in DIAL_CODES}
print(country_code)

print(
    {code: country.upper() for country, code in country_code.items() if code < 66}
)

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62, 'Brazil': 55, 'Pakistan': 92, 'Bangladesh': 880, 'Nigeria': 234, 'Russia': 7, 'Japan': 81}
{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}


### 존재하지 않는 키를 setdefault()로 처리하기

fail-fast 철학에 따라 존재하지 않는 키 k로 d[k]를 접근하면 dict는 오류를 발생시킨다.
keyError를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 d[k] 대신 d.get(k, default)를 사용한다는 것은 파이썬 개발자라면 누구나 알고 있다.

그렇지만 발견한 값을 갱신할 때, 가변 객체면 `__getitem()__`이나 `get()`메서드는 보기 어색하며, 효율성도 떨어진다.

In [22]:
import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no < column_no)
            index.setdefault(word, []).append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

FileNotFoundError: [Errno 2] No such file or directory: '-f'

### 융통성 있게 키를 조회하는 매핑
검색할 떄 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리한 때가 종종 있다.
이런 딕셔너리를 만드는 방법은 크게 두 가지다.
하나는 dict 대신 defaultdict를 사용하는 방법이고, 다른 하나는 dict등의 매핑형을 상속해서 `__missing__()`메서드를 추가하는 방법이다.

In [ ]:
#
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)

# setdefault
my_dict.setdefault(key, []).append(new_value)

# defaultdict
index = collections.defaultdict(list)

### __missing__()메서드
매핑형 이름으로도 쉽게 추측할 수 있는 `__missing__()`메서드를 이용해서 존재하지 않는 키를 처리한다.
이 특수 메서드는 기본 클래스인 dict에는 정의되어 있지 않지만, dict는 이 메서드를 알고있다.

## UserDict 상속하기
`dict` 보다는 `UserDict`를 상속해서 매핑형을 만드는 것이 쉽다.
내장형에서는 아무런 문제없이 상속할 수 있는 메서드들을 오버라이드해야 하는 구현의 특이성 때문에 `dict` 보다는 `UserDict`를 상속하는 것이 낫다.

`UserDict`은 `dict`을 상속하지 않고 내부에 실제 항목을 담고 있는 data라고 하는 dict 객체를 갖고 있다.
이렇게 구현함으써 __setitem__() 등의 특수 메서드를 구현할 때 발생하는 원치않는 재귀적 호출을 피할 수 있으며, `__contains__()`메서드를 간단히 구현할 수 있다.

In [ ]:
# 삽입, 갱신, 조히할 때 비문자열 키를 항상 문자열로 반환하는 StrKeyDict

import collections

class StrKeyDict(collections.UserDict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def __contains__(self, key):
        return str(key) in self.data

    def __setitem__(self, key, item):
        self.data[str(key)] = item

## 불변 매핑

표준 라이브러리에서 제공하는 매핑형은 모두 가변형이지만 사용자가 실수로 매핑을 변경하지 못하도록 보장하고 싶은 경우가 있을 것이다.

파이썬 3.3 이후 types 모듈은 `MappingProxyType`이라는 래퍼 클래스를 제공해서, 원래 매핑의 동적인 뷰를 제공하지만 읽기 전용의 mappingproxy 객체를 반환한다.
따라서 원래 매핑을 변경하면 mappingproxy에 반영되지만, mappingproxy를 직접 변경할 수는 없다.


In [29]:
# dict에서 읽기 전용 mappingproxy 객체를 생성하는 MappingProxyType
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
print(d_proxy)

print(d_proxy[1])
# d_proxy[2] = 'x'

d[2] = 'B'
print(d_proxy)

{1: 'A'}
A
{1: 'A', 2: 'B'}


## 집합 이론

파이썬에서 집합은 비교적 최근에 추가되었으며 그리 많이 사용되지는 않는다. set 형과 set의 불변형 버전인 frozenset은 파이썬2.3에 모듈로 처음 등장했으며, 파이썬2.6에서 내장형으로 승격되었다.


In [30]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)
print(list(set(l)))

['eggs', 'spam']


### 집합 리터럴
{1}, {1, 2} 등 집합 리터럴에 대한 구문은 수학적 표기법과 동일하지만, 공집합은 리터럴로 표기할 수 없으므로, 반드시 set()으로 표기해야한다.

{1, 2, 3}과 같은 리터럴 집합 구문은 set([1, 2, 3])처럼 생성자를 호출하는 것보다 더 빠르고 가독성이 좋다.

생성자를 명시적으로 호출하는 경우에는 파이썬이 생성자를 가져오기 위해 집합명을 검색하고, 리스트릴 생성하고, 이 리스트를 생성자에 전달해야하므로 더 느리다.
반면 {1, 2, 3}과 같은 리터럴 집합 구문을 처리하는 경우, 파이썬을 BULID_SET이라는 특수 바이트 코드를 실행한다.